# Library and Setup

In [229]:
import pandas as pd # Data management tools
import numpy as np # Mathematical operations
import psycopg2 # Access to SQL
import re # Regex and other lookup tools

In [230]:
def fetch_table_to_df(conn, query):
    """"Extracts entire table(s) from an SQL database. 
    
    conn: Should be a connect function from psycopg2.
    query: Tables to be extracted from the database.

    """
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows, columns=column_names).replace({np.nan}, None)
    cur.close()
    return df

In [231]:
conn = psycopg2.connect(
    database="testing",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)

In [232]:
tables = "tree_monitorings", "measurement_informations", "biomass_formulas", "tree_biomasses", 'taxonomies', 'tree_samplings' 
for table in tables:
    table_name = f"{table}"
    globals()[table_name] = fetch_table_to_df(conn, f"SELECT * FROM {table}")

# Biomass Calculation

# Preparation

Python supports $pow()$, only need to remove $ sign from the equation

In [233]:
#replacements = {'pow' : '', ',' : '**'}
#biomass_formulas['formula_python'] = biomass_formulas['formula'].replace(replacements, regex = True).str.replace('$','')

biomass_formulas['formula_python'] = biomass_formulas['formula'].str.replace('$','') #Simpler, pow() exists in python as well

Changing data types for smooth operations

In [234]:
tree_monitorings[['tree_height', 'tree_dbh']] = tree_monitorings[['tree_height', 'tree_dbh']].astype('float')
taxonomies['taxonomy_id'] = taxonomies['taxonomy_id'].astype('int')
taxonomies['wood_density'] = taxonomies['wood_density'].astype('float')
tree_monitorings['taxonomy_id'] = tree_monitorings['taxonomy_id'].astype('int')

Current data shows combination of hasHeight+noDBH, noHeight+noDBH, hasHeight+hasDBH but no noHeight+hasDBH

In [235]:
tree_monitorings[['tree_height','tree_dbh']].notna().value_counts()

tree_height  tree_dbh
True         False       6301
False        False       4193
True         True        3234
Name: count, dtype: int64

Standardizing tree_species names (unfinished, need to deal with NAs)

In [236]:
tree_species = ['Suren', 'Kopi Liberika', 'Citrus', 'Casuarina','Other', 'Meranti','Soursop','Gaharu','Mango','Rosewood','Orange','Tengkurung','Durian','Cajuput','Jackfruit','Rambutan','Clove','Coffee', 'Lamtoro','Meranti Bakau', 'Meranti Bunga', 'Asam Gelugur','Avocado','Cempedak', 'Asam', 'Tampui', 'Sirsak', 'Kuras','Bitterbean','Mentangor']

In [237]:
#x = [next(iter(x), np.nan) 
#          for x in map(lambda x: difflib.get_close_matches(x, tree_species, cutoff = 0.7), tree_monitorings['tree_species']) if x]

## Full Census Biomass Calculation

Adding column of biomass_formula_id for simpler referencing in upcoming operations

In [238]:
dbh_exist = tree_monitorings['tree_dbh'].notna() # check which is not NA
height_exist = tree_monitorings['tree_height'].notna() # check which is not NA

trees_wood_density = [taxonomies.loc[taxonomies['taxonomy_id'] == tax_id, 'wood_density'].values[0] 
                if tax_id in taxonomies['taxonomy_id'].values else None
                for tax_id in tree_monitorings['taxonomy_id']]

In [239]:
for index, (dbh, height, wood) in enumerate(zip(dbh_exist, height_exist, trees_wood_density)):
    if dbh == 1 and height == 1:
        if (wood != wood or wood == None):
            tree_monitorings.loc[index, 'biomass_formulas_id'] = 7
        else:
            tree_monitorings.loc[index, 'biomass_formulas_id'] = 3
    elif (dbh == 1 and height == 0):
        tree_monitorings.loc[index, 'biomass_formulas_id'] = 5
    elif (dbh == 0 and height == 1):
        tree_monitorings.loc[index, 'biomass_formulas_id'] = 4
    else:
        tree_monitorings.loc[index, 'biomass_formulas_id'] = None

In [240]:
tree_monitorings.biomass_formulas_id.value_counts(dropna= False)

biomass_formulas_id
4.0    6301
NaN    4193
7.0    3178
3.0      56
Name: count, dtype: int64

Filtering data from measurement_information which also exist in tree_monitorings

In [241]:
type1_measurement = measurement_informations[(measurement_informations['monitoring_id'].isin(tree_monitorings['id'])== True) & measurement_informations['monitoring_type'] == 1]

In [242]:
tree_monitorings['tree_cond'].value_counts(dropna = False)

tree_cond
0       8011
1       3084
None    2156
2        477
Name: count, dtype: int64

In [243]:
#measurement = measurement_informations[(measurement_informations['monitoring_id'].isin(tree_monitorings['id'])== True)]

Calculation of biomass, still unoptimized

In [244]:
taxonomy_dict = taxonomies.set_index('taxonomy_id')['wood_density'].to_dict() #Set wood identity for referral from taxonomy_id

biomass_index = [] #empty list for storing valid biomass ids
result = [] #empty list for storing results
for index_used in type1_measurement['monitoring_id']:
    tree_dbh     = tree_monitorings.loc[tree_monitorings['id'] == index_used,'tree_dbh'].values[0]
    tree_height  = tree_monitorings.loc[tree_monitorings['id'] == index_used,'tree_height'].values[0]
    biomass_id   = tree_monitorings.loc[tree_monitorings['id'] == index_used,'biomass_formulas_id'].values[0]
    tax_id       = tree_monitorings.loc[tree_monitorings['id'] == index_used,'taxonomy_id'].values[0]
    wood_density = taxonomy_dict.get(tax_id)
    if (biomass_id != biomass_id or biomass_id == None):
        continue
    else:
        biomass_form = biomass_formulas.loc[biomass_formulas['id'] == biomass_id, 'formula_python']
        for biomass in biomass_form:
            biomass_result = eval(biomass)
            result.append(biomass_result)
            biomass_index.append(index_used)
#            print("dbh:", tree_dbh,
#                  "height:", tree_height,
##                  "index:", index_used, 
 #                 "formula", biomass_form,
 #                 "result:", biomass_result)

### Creating Biomass Table/Dataframe

In [245]:
tree_biomasses_python = type1_measurement[type1_measurement['monitoring_id'].isin(biomass_index) == True]

In [246]:
tree_biomasses_python = tree_biomasses_python.drop(['id','timenow','start', 'end','username','notes','_xform_id','_xform_id_string','monitoring_order','month_monitoring','phase'], axis = 1)


In [247]:
tree_biomasses_python['result'] = result

In [248]:
tree_biomasses_python['taxonomy_id'] = [tree_monitorings.loc[tree_monitorings['id'] == mon_id, 'taxonomy_id'].values[0] 
                if mon_id in tree_monitorings['id'].values else None
                for mon_id in tree_biomasses_python['monitoring_id']]

In [249]:
tree_biomasses_python['tree_species'] = [tree_monitorings.loc[tree_monitorings['id'] == mon_id, 'tree_species'].values[0] 
                if mon_id in tree_monitorings['id'].values else None
                for mon_id in tree_biomasses_python['monitoring_id']]

In [250]:
tree_biomasses_python['tree_id'] = [tree_monitorings.loc[tree_monitorings['id'] == mon_id, 'tree_id'].values[0] 
                if mon_id in tree_monitorings['id'].values else None
                for mon_id in tree_biomasses_python['monitoring_id']]

In [251]:
tree_biomasses_python['submission_time'] = [tree_monitorings.loc[tree_monitorings['id'] == mon_id, 'submission_time'].values[0] 
                if mon_id in tree_monitorings['id'].values else None
                for mon_id in tree_biomasses_python['monitoring_id']]

In [252]:
import datetime
tree_biomasses_python['updated_at'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')


In [253]:
carbon = []
for res in result:
    if (res != res or res == None):
        continue
    else:
        carbon.append(res * 0.5 * 3.67) # biomass * 0.5 * 3.67
tree_biomasses_python['carbon'] = carbon

In [254]:
tree_biomasses_python['id'] = range(1,len(tree_biomasses_python)+1)

In [256]:
tree_biomasses_python = tree_biomasses_python[['id','monitoring_id','tree_id','tree_species','taxonomy_id','result','carbon','date_monitoring','submission_time','created_at','updated_at']]
tree_biomasses_python

,id,monitoring_id,tree_id,tree_species,taxonomy_id,result,carbon,date_monitoring,submission_time,created_at,updated_at
1653,1,62,AA01T0180,Meranti,5,0.000000,0.000000,2018-04-23,2019-03-29 14:36:00,2021-08-01 20:19:02,2024-11-18 14:26:27
1686,2,1,AA01T0001,Gaharu,4,0.030735,0.056398,2018-02-21,2019-03-29 14:02:38,2021-08-01 20:19:01,2024-11-18 14:26:27
1793,3,30838,BB-01-0-0024,Suren,64,0.295311,0.541896,2024-08-15,2024-09-11 00:00:00,2024-09-11 18:13:55,2024-11-18 14:26:27
1794,4,30839,BB-01-0-0026,Suren,64,0.144379,0.264936,2024-08-15,2024-09-11 00:00:00,2024-09-11 18:13:55,2024-11-18 14:26:27
1795,5,30840,BB-01-0-0027,Suren,64,0.329164,0.604016,2024-08-15,2024-09-11 00:00:00,2024-09-11 18:13:55,2024-11-18 14:26:27
...,...,...,...,...,...,...,...,...,...,...,...
18791,10581,36980,CL-10-0-1843,suren,64,0.473252,0.868417,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 14:26:27
18792,10582,36981,CL-10-0-1844,suren,64,0.577320,1.059382,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 14:26:27
18793,10583,36982,CL-10-0-1845,suren,64,0.943740,1.731763,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 14:26:27
18794,10584,36983,CL-10-0-1846,suren,64,0.224837,0.412575,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 14:26:27


### Compare result to original table

In [ ]:
#tree_biomasses_python['tree_cond'] = [tree_monitorings.loc[tree_monitorings['id'] == mon_id, 'tree_cond'].values[0] 
#                if mon_id in tree_monitorings['id'].values else None
#                for mon_id in tree_biomasses_python['monitoring_id']]

In [ ]:
#print(tree_biomasses_python['result'].value_counts(dropna= False))

result
0.000000    1504
0.005493     162
0.088504     155
0.009661     142
0.130719     137
            ... 
0.698863       1
0.827691       1
0.540066       1
0.197447       1
0.224837       1
Name: count, Length: 1926, dtype: int64


In [ ]:
#original = []
#new = []
#condition = []
#for index, row in tree_biomasses_python.iterrows():
#    ids = row['monitoring_id']
#    original.append(tree_biomasses.loc[tree_biomasses['monitoring_id'] == ids, 'result'].values[0])
#    new.append(round(row['result'],5))
#    condition.append(tree_monitorings.loc[tree_monitorings['id'] == ids, 'tree_cond'])
#    print(tree_biomasses.loc[tree_biomasses['monitoring_id'] == ids, 'result'].values[0] == round(row['result'],5))

#comp = pd.DataFrame(np.column_stack([original, new, condition]))

In [177]:
tree_biomasses_python

,id,monitoring_id,tree_id,tree_species,taxonomy_id,result,carbon,date_monitoring,submission_time,created_at,updated_at,tree_cond
582,1,506,AA04T0046,KOPI LIBERIKA,7,0.000000,0.000000,2019-06-07,2019-09-06 08:29:18,2021-08-01 19:19:22,2024-11-18 10:45:04,None
583,2,507,AA04T0047,KOPI LIBERIKA,7,0.000000,0.000000,2019-06-07,2019-09-06 08:29:31,2021-08-01 19:19:22,2024-11-18 10:45:04,None
584,3,509,AA04T0049,KOPI LIBERIKA,7,0.000000,0.000000,2019-06-07,2019-09-06 08:29:50,2021-08-01 19:19:22,2024-11-18 10:45:04,None
585,4,510,AA04T0050,KOPI LIBERIKA,7,0.000000,0.000000,2019-06-07,2019-09-06 08:30:00,2021-08-01 19:19:22,2024-11-18 10:45:04,None
586,5,511,AA04T0051,KOPI LIBERIKA,7,0.000000,0.000000,2019-06-07,2019-09-06 08:30:34,2021-08-01 19:19:22,2024-11-18 10:45:04,None
...,...,...,...,...,...,...,...,...,...,...,...,...
18791,11631,36980,CL-10-0-1843,suren,64,0.473252,0.868417,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 10:45:04,0
18792,11632,36981,CL-10-0-1844,suren,64,0.577320,1.059382,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 10:45:04,0
18793,11633,36982,CL-10-0-1845,suren,64,0.943740,1.731763,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 10:45:04,0
18794,11634,36983,CL-10-0-1846,suren,64,0.224837,0.412575,2023-08-21,2024-09-11 00:00:00,2024-09-11 21:22:56,2024-11-18 10:45:04,0


In [178]:
tree_biomasses

,id,monitoring_id,tree_id,tree_species,taxonomy_id,result,carbon,date_monitoring,submission_time,created_at,updated_at
0,1,32192,BB-06-0-1981,Mango,63,0.13072,0.06536,2024-08-15,2024-09-11 00:00:00,2024-09-11 22:42:35,2024-09-11 22:42:35
1,2,1253,P-1-051880062,Jelutung,17,None,None,2019-08-25,2021-07-28 00:00:00,2024-09-11 22:42:35,2024-09-11 22:42:35
2,3,2064,P-1-051880063,Jelutung,17,None,None,2020-09-19,2021-07-28 00:00:00,2024-09-11 22:42:35,2024-09-11 22:42:35
3,4,1254,P-1-051880063,Jelutung,17,None,None,2019-08-25,2021-07-28 00:00:00,2024-09-11 22:42:35,2024-09-11 22:42:35
4,5,2065,P-1-051880064,Jelutung,17,None,None,2020-09-19,2021-07-28 00:00:00,2024-09-11 22:42:35,2024-09-11 22:42:35
...,...,...,...,...,...,...,...,...,...,...,...
10639,10640,37979,CL-10-0-1844,Suren,64,None,None,2024-08-19,2024-09-11 00:00:00,2024-09-11 22:42:55,2024-09-11 22:42:55
10640,10641,37980,CL-10-0-1845,Suren,64,None,None,2024-08-19,2024-09-11 00:00:00,2024-09-11 22:42:55,2024-09-11 22:42:55
10641,10642,37981,CL-10-0-1846,Suren,64,None,None,2024-08-19,2024-09-11 00:00:00,2024-09-11 22:42:55,2024-09-11 22:42:55
10642,10643,37982,CL-10-0-1847,Suren,64,None,None,2024-08-19,2024-09-11 00:00:00,2024-09-11 22:42:55,2024-09-11 22:42:55


In [ ]:
#tree_biomasses_python.tree_cond.value_counts(dropna= False)

tree_cond
0       8008
None    3150
2        477
Name: count, dtype: int64

In [168]:
#comp[2].value_counts(dropna= False)

2
0       8008
None    3150
2        477
Name: count, dtype: int64

## Sampling Biomass Calculation

In [187]:
sampling_dbh_exist = tree_samplings['tree_dbh'].notna() # check which is not NA
sampling_height_exist = tree_samplings['tree_height'].notna() # check which is not NA

sampling_trees_wood_density = [taxonomies.loc[taxonomies['taxonomy_id'] == tax_id, 'wood_density'].values[0] 
                if tax_id in taxonomies['taxonomy_id'].values else None
                for tax_id in tree_samplings['taxonomy_id']]

In [188]:
for index, (dbh, height, wood) in enumerate(zip(sampling_dbh_exist, sampling_height_exist, sampling_trees_wood_density)):
    if dbh == 1 and height == 1:
        if (wood != wood or wood == None):
            tree_samplings.loc[index, 'biomass_formulas_id'] = 7
        else:
            tree_samplings.loc[index, 'biomass_formulas_id'] = 3
    elif (dbh == 1 and height == 0):
        tree_samplings.loc[index, 'biomass_formulas_id'] = 5
    elif (dbh == 0 and height == 1):
        tree_samplings.loc[index, 'biomass_formulas_id'] = 4
    else:
        tree_samplings.loc[index, 'biomass_formulas_id'] = None

In [189]:
tree_samplings.biomass_formulas_id.value_counts(dropna= False)

biomass_formulas_id
4       11467
None     2378
7           2
Name: count, dtype: int64

# Data Analytics

In [195]:
data_analytics = fetch_table_to_df(conn, "SELECT * FROM data_analytics")

In [197]:
data_analytics.key.unique()

array(['rewetting_area', 'cseq_revegetation_annualy',
       'cseq_revegetation_accumulative', 'revegetation_area', 'peat_area',
       'cseq_rewetting_annualy', 'cseq_rewetting_accumulative',
       'mineral_area', 'avg_growth_dbh', 'num_tree_species',
       'avg_growth_height', 'subsidence_a_all', 'subsidence_a',
       'subsidence_c', 'subsidence_c_all', 'restoration_area'],
      dtype=object)

In [198]:
survival_rates = fetch_table_to_df(conn, "SELECT * FROM survival_rates")

In [204]:
survival_rates.tree_id.value_counts()

tree_id
P-1-031880009    6
P-1-051890076    6
P-1-061890120    6
P-1-061880092    6
P-1-041880032    6
                ..
AA06T0048        1
AA06T0049        1
AA06T0050        1
AA06T0051        1
CL-02-0-0728     1
Name: count, Length: 5127, dtype: int64

In [221]:
survival_rates[survival_rates.tree_id.isin(tree_monitorings['tree_id'])]

,id,site_id,plot_id,tree_id,tree_species,taxonomy_id,date_monitoring,value,created_at,updated_at
0,1,P,P-1,P-1-031880009,Belangeran,18,2018-12-12,100.00,2024-09-11 22:52:56,2024-09-11 22:52:56
1,2,P,P-1,P-1-031880009,Belangeran,18,2019-05-26,100.00,2024-09-11 22:52:56,2024-09-11 22:52:56
2,3,P,P-1,P-1-031880009,Belangeran,18,2019-08-25,100.00,2024-09-11 22:52:56,2024-09-11 22:52:56
3,4,P,P-1,P-1-031880009,Belangeran,18,2019-12-22,100.00,2024-09-11 22:52:56,2024-09-11 22:52:56
4,5,P,P-1,P-1-031880009,Belangeran,18,2020-09-19,100.00,2024-09-11 22:52:56,2024-09-11 22:52:56
...,...,...,...,...,...,...,...,...,...,...
10508,10509,BB,BB-03,BB-03-0-0762,rosewood,85,2023-08-22,100.00,2024-09-11 22:53:23,2024-09-11 22:53:23
10509,10510,BB,BB-03,BB-03-0-0762,,0,2024-08-15,0.00,2024-09-11 22:53:23,2024-09-11 22:53:23
10510,10511,BB,BB-03,BB-03-0-0995,suren,64,2022-09-29,100.00,2024-09-11 22:53:23,2024-09-11 22:53:23
10511,10512,BB,BB-03,BB-03-0-0995,suren,64,2023-08-22,100.00,2024-09-11 22:53:23,2024-09-11 22:53:23


In [223]:
measurement_informations.monitoring_type.value_counts(dropna= False)

monitoring_type
1    13730
2     3217
3     1849
Name: count, dtype: int64

In [ ]:
plot(measurement_informations.groupby('monitoring_type')['monitoring_id']

,count,mean,std,min,25%,50%,75%,max
monitoring_type,,,,,,,,
1,13730.0,27049.827167,11995.755931,1.0,27685.25,31117.5,34549.75,37982.0
2,3217.0,1609.134287,929.034629,1.0,805.00,1609.0,2413.00,3220.0
3,1849.0,925.000000,533.904642,1.0,463.00,925.0,1387.00,1849.0


In [224]:
measurement_informations[measurement_informations['monitoring_type'] == 1].describe()

,id,monitoring_id,monitoring_type,created_at,updated_at,_xform_id,monitoring_order,phase
count,13730.000000,13730.000000,13730.0,13730,13730,13730.000000,13730.000000,13730.0
mean,35888.686016,27049.827167,1.0,2024-03-15 14:14:27.047268608,2024-03-23 21:23:01.403495936,37.139548,1.456519,1.0
min,1.000000,1.000000,1.0,2021-07-28 19:51:03,2021-07-28 19:51:03,0.000000,0.000000,1.0
25%,38312.250000,27685.250000,1.0,2024-09-11 18:13:04,2024-09-11 18:13:04,0.000000,1.000000,1.0
50%,41744.500000,31117.500000,1.0,2024-09-11 18:13:56,2024-09-11 18:13:56,0.000000,1.000000,1.0
75%,45176.750000,34549.750000,1.0,2024-09-11 21:21:54,2024-09-11 21:21:54,0.000000,2.000000,1.0
max,48609.000000,37982.000000,1.0,2024-09-11 21:23:26,2024-09-11 21:23:26,1249.000000,3.000000,1.0
std,15874.015029,11995.755931,0.0,NaN,NaN,129.660914,0.956347,0.0


In [ ]:
conn.close()

x